In [4]:
import yfinance as yf
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import VBox, HBox, Layout

def fetch_financial_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        financials = stock.financials
        financials.columns = pd.to_datetime(financials.columns, format='%Y-%m-%d', errors='coerce')
        financials = financials.dropna(axis=1, how='all')
        financials = financials.sort_index(axis=1)
        return financials
    except Exception as e:
        return None

def calculate_growth(data):
    growth = {}
    for i in range(len(data) - 1):
        current_year = data.index[i]
        next_year = data.index[i + 1]
        if pd.isna(data[current_year]) or pd.isna(data[next_year]) or data[current_year] == 0:
            growth[next_year] = None
        else:
            growth[next_year] = ((data[next_year] - data[current_year]) / data[current_year]) * 100
    return growth

def generate_summary(ticker, start_date, end_date):
    financials = fetch_financial_data(ticker)
    if financials is None or financials.empty:
        return "Error fetching financial data or data is unavailable."

    summary = f"\nSummary for {ticker} from {start_date.date()} to {end_date.date()}:\n"

    try:
        def get_financial_data(row_name):
            if row_name in financials.index:
                return pd.to_numeric(financials.loc[row_name].fillna(0), errors='coerce')
            return pd.Series(dtype=float)

        net_profit = get_financial_data("Net Income")
        gross_profit = get_financial_data("Gross Profit")
        operating_profit = get_financial_data("Operating Income")

        available_dates = financials.columns
        if start_date not in available_dates or end_date not in available_dates:
            available_years = sorted(set(available_dates.year))
            available_dates_str = ", ".join(available_dates.strftime('%Y-%m-%d'))
            return (
                f"Error: Data for the selected date range is unavailable.\n"
                f"Available years: {', '.join(map(str, available_years))}.\n"
                f"Exact available dates: {available_dates_str}."
            )

        net_profit = net_profit[start_date:end_date]
        gross_profit = gross_profit[start_date:end_date]
        operating_profit = operating_profit[start_date:end_date]

        net_growth = calculate_growth(net_profit)
        gross_growth = calculate_growth(gross_profit)
        operating_growth = calculate_growth(operating_profit)

        summary += "Net Profit Growth Year-over-Year:\n"
        for year, growth in net_growth.items():
            summary += f"  {year}: {growth:.2f}%\n" if growth is not None else f"  {year}: Data Unavailable\n"

        summary += "\nGross Profit Growth Year-over-Year:\n"
        for year, growth in gross_growth.items():
            summary += f"  {year}: {growth:.2f}%\n" if growth is not None else f"  {year}: Data Unavailable\n"

        summary += "\nOperating Profit Growth Year-over-Year:\n"
        for year, growth in operating_growth.items():
            summary += f"  {year}: {growth:.2f}%\n" if growth is not None else f"  {year}: Data Unavailable\n"

    except KeyError as e:
        summary = f"Error accessing key financial data: {e}"
    except Exception as e:
        summary = f"An error occurred: {e}"

    return summary

def on_submit(change):
    ticker = ticker_input.value
    start_date = start_date_input.value
    end_date = end_date_input.value

    if not ticker:
        result_output.value = "Please provide a valid ticker symbol."
        return

    try:
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
    except ValueError:
        result_output.value = "Please enter valid dates for start and end in YYYY-MM-DD format."
        return

    summary = generate_summary(ticker, start_date, end_date)
    result_output.value = summary

def create_gradient_label(text):
    return widgets.HTML(
        value=f"""<div style='background: linear-gradient(to right, #6a11cb, #2575fc); color: white;
                        padding: 10px; text-align: center; border-radius: 5px; font-size: 18px;'>
                        {text}</div>"""
    )

ticker_input = widgets.Text(
    description="Ticker:",
    style={'description_width': '100px'},
    layout=Layout(width='300px', margin='5px 0')
)
start_date_input = widgets.Text(
    description="Start Date:",
    placeholder="YYYY-MM-DD",
    style={'description_width': '100px'},
    layout=Layout(width='300px', margin='5px 0')
)
end_date_input = widgets.Text(
    description="End Date:",
    placeholder="YYYY-MM-DD",
    style={'description_width': '100px'},
    layout=Layout(width='300px', margin='5px 0')
)
submit_button = widgets.Button(
    description="Submit",
    button_style="success",
    layout=Layout(width='150px', margin='10px 0')
)
result_output = widgets.Textarea(
    value="",
    placeholder="Results will be displayed here.",
    description="",
    layout=Layout(width='600px', height='400px', margin='10px', padding='10px', border='2px solid #2575fc', border_radius='5px')
)

submit_button.on_click(on_submit)

header_label = create_gradient_label("Profit Growth Analysis Tool By Deep FinOne")
input_section = VBox([
    ticker_input,
    start_date_input,
    end_date_input,
    submit_button
], layout=Layout(align_items='flex-start', padding='10px'))
output_section = VBox([
    create_gradient_label("Output"),
    result_output
], layout=Layout(padding='10px'))

app_layout = VBox([
    header_label,
    HBox([input_section], layout=Layout(justify_content='flex-start')),
    output_section
], layout=Layout(padding='20px', border='solid 2px #ccc', border_radius='10px'))

display(app_layout)